<a href="https://colab.research.google.com/github/won-hj/testKeras/blob/main/Object_Detection_with_KerasCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Object Detection with KerasCV
-
KerasCV offers a complete set of production grade APIs to solve object detection problems. These APIs include object-detection-specific data augmentation techniques, Keras native COCO metrics, bounding box format conversion utilities, visualization tools, pretrained object detection models, and everything you need to train your own state of the art object detection models!
+
KerasCV는 객체 감지 문제를 해결하기 위해 완전한 프로덕션 등급 API 세트를 제공합니다. 이러한 API에는 객체 감지별 데이터 증대 기술, Keras 기본 COCO 측정항목, 경계 상자 형식 변환 유틸리티, 시각화 도구, 사전 학습된 객체 감지 모델 및 자체 최첨단 객체 감지 모델을 학습하는 데 필요한 모든 것이 포함되어 있습니다.

🔗 https://keras.io/guides/keras_cv/object_detection_keras_cv/


In [1]:
'''
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.
'''
!pip install -q  --upgrade keras-cv
!pip install -q --upgrade keras #upgrade keras3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax" # @param ["tensorflow", "jax", "torch"]

from tensorflow import data as tf_data
import tensorflow_datasets as tfds
import keras
import keras_cv
import numpy as np
from keras_cv import bounding_box
import os
from keras_cv import visualization
import tqdm

#Object detection introduction
- Object detection is the process of identifying, classifying, and localizing objects within a given image. Typically, your inputs are images, and your labels are bounding boxes with optional class labels. Object detection can be thought of as an extension of classification, however instead of one class label for the image, you must detect and localize an arbitrary number of classes.
+ 객체 감지는 주어진 이미지 내에서 객체를 식별, 분류 및 위치화하는 프로세스입니다. 일반적으로 입력은 이미지이고 레이블은 선택적 클래스 레이블이 있는 경계 상자입니다. 객체 감지는 분류의 확장으로 생각할 수 있지만 이미지에 대한 하나의 클래스 레이블 대신 임의 개수의 클래스를 감지하고 지역화해야 합니다.

#Perform detections with a pretrained model
- The highest level API in the KerasCV Object Detection API is the keras_cv.models API. This API includes fully pretrained object detection models, such as keras_cv.models.YOLOV8Detector.
Let's get started by constructing a YOLOV8Detector pretrained on the pascalvoc dataset.
- KerasCV 객체 감지 API의 최고 수준 API는 API입니다 keras_cv.models. 이 API에는 다음과 같이 완전히 사전 훈련된 객체 감지 모델이 포함되어 있습니다 keras_cv.models.YOLOV8Detector.
데이터세트 에 사전 훈련된 YOLOV8Detector를 구성하는 것부터 시작해 보겠습니다 pascalvoc .

 🔗 YOLO https://keras.io/api/keras_cv/models/tasks/yolo_v8_detector#yolov8detector-class

In [ ]:
pretrained_model = keras_cv.models.YOLOV8Detector.from_preset("yolo_v8_m_pascalvoc", bounding_box_format="xywh")

- bounding_box_format


    bounding_boxes = {
      "classes": [num_boxes],
      "boxes": [num_boxes, 4]
    }

이 인수는 파이프라인에서 레이블 사전 필드 의 값이 사용하는 형식을 정확하게 설명합니다. 예를 들어, 왼쪽 위 모서리가 좌표 (100, 100)에 있고 너비가 55이고 높이가 70인 형식 "boxes"의 상자 는 다음과 같이 표시됩니다.

 ➡ xywh
    
    [100, 100, 55, 75]

➡ xyxy

    [100, 100, 155, 175]

간단해 보일 수도 있지만 KerasCV 객체 감지 API의 중요한 부분입니다! 경계 상자를 처리하는 모든 구성 요소에는 bounding_box_format인수가 필요합니다. API 문서에서 KerasCV 경계 상자 형식에 대한 자세한 내용을 읽을 수 있습니다 .

이는 경계 상자에 대한 올바른 형식이 없기 때문에 수행됩니다! 서로 다른 파이프라인의 구성요소는 서로 다른 형식을 예상하므로 이를 지정하도록 요구함으로써 구성요소가 읽기 쉽고 재사용 가능하며 명확하게 유지되도록 보장합니다. 상자 형식 변환 버그는 아마도 객체 감지 파이프라인에서 가장 일반적인 버그 표면일 것입니다. 이 매개변수를 요구함으로써 우리는 이러한 버그를 완화합니다(특히 여러 소스의 코드를 결합할 때).

🔗https://keras.io/api/keras_cv/bounding_box/formats/

In [7]:
filepath = keras.utils.get_file(origin="https://i.imgur.com/gCNcJJI") #error;%d format: a real number is required, not NoneType
image = keras.utils.load_img(filepath)
image = np.array(image)

visualization.plot_image_gallery(
    np.array([image]),
    value_range = (0, 255),
    row=1,
    cols=1,
    scale=5,
)

      0/Unknown 0s 0s/step

TypeError: %d format: a real number is required, not NoneType

In [6]:
import traceback
print(traceback.format_exc()) #NoneType: None


NoneType: None

